In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
from scipy.optimize import root
from matplotlib.font_manager import FontProperties

font_path = r'C:\Windows\Fonts\simsun.ttc'
chinese_font = FontProperties(fname=r'C:\Windows\Fonts\simsun.ttc')
english_font = FontProperties(fname=r'C:\Windows\Fonts\times.ttf')

rural = np.arange(0, 60, 2)
urban = np.arange(1, 60, 2)
east = np.r_[0, 1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 24, 25, 28, 29, 36, 37, 40, 41]
middle = np.r_[6, 7, 22, 23, 26, 27, 30, 31, 32, 33, 34, 35]
west = np.r_[8, 9, 10, 11, 12, 13, 14, 15, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]

In [2]:
# import data and parameters

equilibrium2010 = np.load('./data/equilibrium2010.npz')
R = equilibrium2010['R']
X = equilibrium2010['X']
m = equilibrium2010['m']
pi = equilibrium2010['pi']
L = equilibrium2010['L']
L_bar = equilibrium2010['L_bar']
K = equilibrium2010['K']
w = equilibrium2010['w']
P = equilibrium2010['P']
W = equilibrium2010['W']

parameter = np.load('./data/Parameter.npz')
kappa = parameter['kappa']
theta = parameter['theta']
beta = parameter['beta']
psi_ag = parameter['psi_ag']
psi_na = parameter['psi_na']
alpha_ag = parameter['alpha_ag']
alpha_na = parameter['alpha_na']
sigma_ag = parameter['sigma_ag']
sigma_na = parameter['sigma_na']
eta_ag_ag = parameter['eta_ag_ag']
eta_ag_na = parameter['eta_ag_na']
eta_na_ag = parameter['eta_na_ag']
eta_na_na = parameter['eta_na_na']
phiu = parameter['phiu']
phip = parameter['phip']
chi = parameter['chi']
eta = parameter['eta']
delta = parameter['delta']
xi_ag = parameter['xi_ag']
xi_na = parameter['xi_na']
t = parameter['t']

In [3]:
# import endogenous and exogenous variables

## keep exogenous variables unchanged
dKp = np.ones([60, 1])
dKh = np.ones([60, 1])
dT = np.ones([60, 1])
dtau = np.ones([60, 60])
dmu = np.ones([60, 60])

In [4]:
# define a function to solve the counterfactural equilibrium
## in the following blocks, different parameters are used to simulate different outcomes after the land reform
## this block is similar to the file "2.Counterfactural"

def SolveCounter(XX):
    dw = XX[:60].reshape([60, 1])
    dL = XX[60:120].reshape([60, 1])
    dp = XX[120:].reshape([60, 1])

    ## (a) change in price index
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))

    ## (b) change in public expenditures
    dG = dw * dL / dP

    ## (c) change in the cost of the unit input bundle
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p

    ## (d) change in the trade share
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta))
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1])
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi

    ## (e) change in the price of final goods
    dp1 = dpi_den ** (-1 / theta)

    ## (f) change in the income ratio
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    w = wL / L 
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1])
    rho_prime = rKrK * dw * dL / L_bar
    rho_prime[1::2] = 0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1])
    ddelta = delta_prime
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag)

    ## (g) change in the migration share
    dg = dG / (dL ** chi) 
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa 
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) 
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm

    ## (h) change in the number of residents
    dL1 = (m_prime @ L_bar) / L

    ## (i) total outputs and expenditures under counterfactural equilibrium
    R_prime = R * dw * dL
    wL_prime = wL * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime 
    vL_prime = (wL_prime.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG_prime = (wL_prime.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A_prime = (beta * (1 - t) * np.repeat(vL_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C_prime = (np.repeat(PG_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag_prime = ((R_prime.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na_prime = ((R_prime.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_prime = np.concatenate([B_ag_prime, B_na_prime], axis=1).reshape([60, 1])
    X_prime1 = A_prime + B_prime + C_prime 

    ## (j) conditions under the equilibrium
    res1 = dp1 - dp
    res2 = dL1 - dL
    res3 = X_prime1 - X_prime

    return np.concatenate([res1, res2, res3]).flatten().astype(float)

In [5]:
# define the function to save variables

def counterfautual_result(dw, dL, dp):
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))
    dG = dw * dL / dP
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta))
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) 
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi 
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    w = wL / L 
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1]) 
    rho_prime = rKrK * dw * dL / L_bar 
    rho_prime[1::2] = 0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1])
    ddelta = delta_prime 
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag)
    dg = dG / (dL ** chi)
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1])
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm
    R_prime = R * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime

    return dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, delta_diag

In [6]:
XX = root(SolveCounter, x0=np.ones(3*60), method='lm')
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

In [7]:
# define the function to calculate welfare

def welfare():
    coef_g_na = 1 - beta + sigma_na / alpha_na + beta * t
    coef_g_ag = (alpha_ag + sigma_ag) * t / (beta + t - beta * t) / alpha_ag
    g = ((w * L).reshape([30, 2]) * np.array([coef_g_ag, coef_g_na]).reshape([1, 2])).reshape([60, 1]) / P / (L ** chi)
    
    dV = dW * ((dg) ** phiu) * (np.diag(dm).reshape([-1, 1]) ** (-1 / kappa)) * (np.diag(ddelta).reshape([-1, 1]))

    weight = (np.diag(m).reshape([-1, 1]) ** (-1 / kappa)) * (g ** phiu) * W * L_bar * delta_diag
    weight_country = weight / weight.sum()
    weight_urban = weight[urban] / weight[urban].sum()
    weight_rural = weight[rural] / weight[rural].sum()
    weight_east = weight[east] / weight[east].sum()
    weight_middle = weight[middle] / weight[middle].sum()
    weight_west = weight[west] / weight[west].sum()

    dV_country = ((dV * weight_country).sum() - 1) * 100
    dV_urban = ((dV[urban] * weight_urban).sum() - 1) * 100
    dV_rural = ((dV[rural] * weight_rural).sum() - 1) * 100
    dV_east = ((dV[east] * weight_east).sum() - 1) * 100
    dV_middle = ((dV[middle] * weight_middle).sum() - 1) * 100
    dV_west = ((dV[west] * weight_west).sum() - 1) * 100
    
    return dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west

In [9]:
# baseline welfare

XX = root(SolveCounter, x0=np.ones(3*60), method='lm')
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west = welfare()

table_baseline = pd.DataFrame([dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west]).T.round(3)
table_baseline.columns = ['whole country', 'urban', 'rural', 'east', 'middle', 'west']
table_baseline

whole country  urban  rural   east  middle   west
0          1.664 -1.689  5.108  2.331   1.468  0.698

In [ ]:
# rivalness of public goods: non-rival, chi=0

chi = 0
XX = root(SolveCounter, x0=np.ones(3*60), method='lm') 
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime,delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west = welfare()

table_chi0 = pd.DataFrame([dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west]).T.round(3)
table_chi0.columns = ['whole country', 'urban', 'rural', 'east', 'middle', 'west']
table_chi0

whole country  urban  rural   east  middle   west
0          1.619 -1.354   4.82  2.175   1.473  0.775

In [ ]:
# rivalness of public goods: totally-rival, chi=1

chi = 1
XX = root(SolveCounter, x0=np.ones(3*60), method='lm')
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime,delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west = welfare()

table_chi1 = pd.DataFrame([dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west]).T.round(3)
table_chi1.columns = ['whole country', 'urban', 'rural', 'east', 'middle', 'west']
table_chi1

whole country  urban  rural   east  middle   west
0          1.673 -1.722  5.144  2.358   1.468  0.689

In [13]:
# housing price effects: Lower housing expenditure

chi = 0.9 ## set chi back
beta = 0.99
XX = root(SolveCounter, x0=np.ones(3*60), method='lm')
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime,delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west = welfare()

table_beta99 = pd.DataFrame([dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west]).T.round(3)
table_beta99.columns = ['whole country', 'urban', 'rural', 'east', 'middle', 'west']
table_beta99

whole country  urban  rural   east  middle   west
0          1.469 -1.345  4.567  1.992   1.253  0.768

In [14]:
# housing price effects: higher housing expenditure

beta = 0.8
XX = root(SolveCounter, x0=np.ones(3*60), method='lm')
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])
dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime,delta_diag = counterfautual_result(dw, dL, dp)
W = w / P
w_prime = w * dw
L_prime = L * dL
P_prime = P * dP
W_prime = w_prime / P_prime

dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west = welfare()

table_beta80 = pd.DataFrame([dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west]).T.round(3)
table_beta80.columns = ['whole country', 'urban', 'rural', 'east', 'middle', 'west']
table_beta80

whole country  urban  rural   east  middle   west
0            2.0 -2.262  5.985  2.922   1.789  0.612

In [ ]:
# Table 3

table = pd.DataFrame() 
table = table.append(
        [table_baseline, table_chi0, table_chi1, table_beta99, table_beta80], 
    ignore_index=True)
table.index = ['baseline', 'chi=0', 'chi=1', 'beta=0.99', 'beta=0.8']
table

whole country  urban  rural   east  middle   west
baseline           1.664 -1.689  5.108  2.331   1.468  0.698
chi=0              1.619 -1.354  4.820  2.175   1.473  0.775
chi=1              1.673 -1.722  5.144  2.358   1.468  0.689
beta=0.99          1.469 -1.345  4.567  1.992   1.253  0.768
beta=0.8           2.000 -2.262  5.985  2.922   1.789  0.612